# MMA Fighter Tracker - Interactive Demo

This notebook demonstrates the MMA Fighter Tracker system step by step, showing how to:
- Load and visualize video frames
- Run the tracker on MMA footage
- Analyze tracking results
- Visualize fighter trajectories

## Prerequisites

Make sure you have installed all dependencies:
```bash
pip install -r requirements.txt
```


In [ ]:
# Import required libraries
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import subprocess
import torch
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")


Libraries imported successfully!


## 1. Setup and Video Loading

First, let's check if we have the example video and set up our workspace.


In [3]:
# Check for example video
example_video = "../example_data/UFC_20250803_Dricus_Du_Plessis_vs_Israel_Adesanya_FULL_FIGHT_UFC_319.f616 - chunk_17 [510-540]s.mp4"

if os.path.exists(example_video):
    print("Example video found!")
    
    # Get video properties
    cap = cv2.VideoCapture(example_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps
    
    print(f"Video Properties:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps:.2f}")
    print(f"   Duration: {duration:.1f} seconds")
    print(f"   Total frames: {frame_count}")
    
    cap.release()
else:
    print("Example video not found!")
    print("Please ensure the example video is in the example_data/ directory")


Example video found!
Video Properties:
   Resolution: 1920x1080
   FPS: 29.97
   Duration: 30.0 seconds
   Total frames: 899


## 2. Run the Tracker

Now let's run the MMA Fighter Tracker on our video. This will process the video and generate tracking results.


In [6]:
import torch

# Check for GPU availability and print status
if torch.cuda.is_available():
    device = "cuda"
    print("GPU detected! Using CUDA for model inference.")
else:
    device = "cpu"
    print("GPU NOT detected. Running on CPU (tracking may be slower).")

# Set up output directory
output_dir = "../demo_results"
os.makedirs(output_dir, exist_ok=True)

# Run the tracker with GPU enabled if available
print("Running MMA Fighter Tracker...")
print("This may take a few minutes depending on your hardware...")

cmd = [
    "python", "../src/tracker.py",
    "--video", example_video,
    "--outdir", output_dir,
    "--max-frames", "150",  # Process first 150 frames for demo
    "--stride", "5",        # Process every 5th frame
    "--conf", "0.5",        # Detection confidence threshold
    "--device", device      # Specify device: 'cuda' if GPU, else 'cpu'
]

try:
    result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
    
    if result.returncode == 0:
        print("Tracker completed successfully!")
        print("\nTracker Output:")
        print(result.stdout)
    else:
        print("Tracker failed!")
        print("Error:", result.stderr)
        
except subprocess.TimeoutExpired:
    print("Tracker timed out after 5 minutes")
except Exception as e:
    print(f"Error running tracker: {e}")


GPU detected! Using CUDA for model inference.
Running MMA Fighter Tracker...
This may take a few minutes depending on your hardware...
Tracker completed successfully!

Tracker Output:
Device: cuda
Video: 1920x1080 @ 29.97fps, 899 frames
Loading YOLOv8-seg...
Loading BotSORT tracker...
[VIPS-ReID] Loading osnet_x1_0...
Successfully loaded imagenet pretrained weights from "C:\Users\alagu/.cache\torch\checkpoints\osnet_x1_0_imagenet.pth"
[VIPS-ReID] OSNet loaded successfully!
Config: conf=0.5, iou=0.7, max_persons=3
VIPS ReID: ENABLED

Processing...
  Frame 95: 2 active tracks
  Frame 195: 1 active tracks
  Frame 295: 3 active tracks
  Frame 395: 3 active tracks
  Frame 495: 3 active tracks
  Frame 595: 3 active tracks
  Frame 695: 3 active tracks

Done! Processed 150 frames
  Video: ../demo_results\tracked.mp4
  CSV: ../demo_results\tracks.csv
  Labels: ../demo_results\labels



## 3. Analyze Results

Let's load the tracking results and analyze what the system detected.


In [7]:
# Load tracking results
tracks_file = os.path.join(output_dir, "tracks.csv")

if os.path.exists(tracks_file):
    df = pd.read_csv(tracks_file)
    
    print("Tracking Results Summary:")
    print(f"   Total detections: {len(df)}")
    print(f"   Unique track IDs: {df['track_id'].nunique()}")
    print(f"   Frame range: {df['frame'].min()} - {df['frame'].max()}")
    
    # Track distribution
    track_counts = df['track_id'].value_counts().sort_index()
    print("\nTrack ID Distribution:")
    for track_id, count in track_counts.items():
        print(f"   Track {track_id}: {count} detections")
    
    # Frames with multiple fighters
    fighters_per_frame = df.groupby('frame')['track_id'].count()
    frames_with_2 = (fighters_per_frame == 2).sum()
    frames_with_1 = (fighters_per_frame == 1).sum()
    
    print(f"\nFighter Visibility:")
    print(f"   Frames with 2 fighters: {frames_with_2}")
    print(f"   Frames with 1 fighter: {frames_with_1}")
    
    # Display first few rows
    print("\nSample Data:")
    print(df.head(10))
    
else:
    print("No tracking results found!")
    print("Make sure the tracker ran successfully.")


Tracking Results Summary:
   Total detections: 353
   Unique track IDs: 7
   Frame range: 0 - 745

Track ID Distribution:
   Track 1: 128 detections
   Track 2: 23 detections
   Track 3: 80 detections
   Track 4: 37 detections
   Track 5: 34 detections
   Track 7: 13 detections
   Track 8: 38 detections

Fighter Visibility:
   Frames with 2 fighters: 43
   Frames with 1 fighter: 27

Sample Data:
   frame  track_id     x1     y1      x2      y2
0      0         1  559.0  256.0  1420.0  1015.0
1      5         1  823.0  305.0  1404.0  1034.0
2     10         1  748.0  362.0  1366.0  1047.0
3     15         1  447.0  334.0  1331.0  1068.0
4     20         1  530.0  299.0  1314.0  1064.0
5     25         1  415.0  263.0  1292.0  1062.0
6     30         1  571.0  243.0  1304.0  1036.0
7     35         1  645.0  236.0  1309.0  1035.0
8     40         1  676.0  238.0  1379.0  1050.0
9     45         1  689.0  241.0  1418.0  1044.0
